# Examples

This is a short notebook to illustrate some of the key features of the `precession` module. There much more than this in the code but hopefully this is a good starting point. 

In [ ]:
import numpy as np
from importlib.machinery import SourceFileLoader
precession = SourceFileLoader("precession", "../precession/precession.py").load_module()

## 1. Select a binary configuration.
First, set the constants of motion. These include mass ratio and spin magnitudes:

In [ ]:
q = 0.8
q

In [ ]:
chi1 = 0.5
chi1

In [ ]:
chi2 = 0.9
chi2

From these, find a suitable value of the effective spin within its geometrical limits

In [ ]:
chieff_minus,chieff_plus=precession.chiefflimits(q=q, chi1=chi1, chi2=chi2)
chieff = 0.1
assert chieff>=chieff_minus and chieff<=chieff_plus
chieff

Now the quantities that vary on the radiation-reaction timescale. Specify the orbital separation

In [ ]:
r = 1000
r

Find a suitable value of the asymptotic angular momentum (its boundaries are the spin-orbit resonances):

In [ ]:
kappa_minus,kappa_plus=precession.kappalimits(r=r,chieff=chieff, q=q, chi1=chi1, chi2=chi2)  
kappa = 0.02
assert kappa>=kappa_minus and kappa<=kappa_plus

This is more conventiently done by specifying a dimensionless number, e.g.

In [ ]:
kappa = precession.kapparescaling(kappatilde=0.5, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
kappa

Finally, the precession-timescale variation is encoded in the weighted spin difference. Its limits are given by the solutions of a cubic polynomial:

In [ ]:
deltachi_minus,deltachi_plus = precession.deltachilimits(kappa=kappa,r=r,chieff=chieff,q=q,chi1=chi1,chi2=chi2)
deltachi=-0.1
assert deltachi>=deltachi_minus and deltachi<=deltachi_plus

You can also sample deltachi from its PN-weighted porability density function

In [ ]:
deltachi = precession.deltachisampling(kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi1, N=1)

Or, as before, specify a dimensionless number, e.g.

In [ ]:
deltachi = precession.deltachirescaling(deltachitilde=0.4, kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
deltachi

From these quantities, one can compute the spin angles

In [ ]:
theta1, theta2, deltaphi = precession.conserved_to_angles(deltachi=deltachi, kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
theta1,theta2,deltaphi

The inverse operation is

In [ ]:
deltachi_, kappa_, chieff_ = precession.angles_to_conserved(theta1=theta1, theta2=theta2, deltaphi=deltaphi, r=r, q=q, chi1=chi1, chi2=chi2)
assert np.isclose(deltachi_,deltachi) and np.isclose(kappa_,kappa) and np.isclose(chieff_,chieff)

One can now compute various derived quantities including:
 
* The precession period
* The total precession angle
* Two flavors of the precession estimator chip
* The spin morphology
* ... and many more!


In [ ]:
tau = precession.eval_tau(kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
tau

In [ ]:
alpha = precession.eval_alpha(kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
alpha

In [ ]:
chip_averaged = precession.eval_chip(deltachi=deltachi,kappa=kappa,r=r,chieff=chieff,q=q,chi1=chi1,chi2=chi2,which="averaged")
chip_averaged

In [ ]:
chip_rms = precession.eval_chip(deltachi=deltachi,kappa=kappa,r=r,chieff=chieff,q=q,chi1=chi1,chi2=chi2,which="rms")
chip_rms

In [ ]:
morph = precession.morphology(kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2 )
morph

## 2. Inspiral

Let's now evolve that binary configuration down to r=10M. 

In [ ]:
rvals = [r, 10] # Specify all the outputs you want here
outputs = precession.inspiral_precav(kappa=kappa,r=rvals,chieff=chieff,q=q,chi1=chi1,chi2=chi2)
outputs

The same evolution can also be done with an orbit-averaged scheme (note that now you have to specify deltachi as well). This is slow:

In [ ]:
outputs = precession.inspiral_orbav(deltachi=deltachi, kappa=kappa,r=rvals,chieff=chieff,q=q,chi1=chi1,chi2=chi2)
outputs

Let's now take this binary at r=10M, and propagate it back to past time infinity:

In [ ]:
rvals = [outputs['r'][0,-1], np.inf] # Specify all the outputs you want here
outputs = precession.inspiral_precav(kappa=outputs['kappa'][0,-1],r=rvals,chieff=chieff,q=q,chi1=chi1,chi2=chi2)
outputs

## 3. Precession average what you like

The code also allow you to precession-average quantities specified by the users. For instance, let's compute the average of deltachi. 

In [ ]:
def func(deltachi):
    return deltachi

deltachiav_ = precession.precession_average(kappa=kappa, r=r, chieff=chieff, q=q, chi1=chi1, chi2=chi2, func=func)
deltachiav_

For this specific case, the averaged can also be done semi-analytically in terms of elliptic integrals (though in general this is not possible). Let's check we get the same result:

In [ ]:
u = precession.eval_u(r, q)
deltachiminus,deltachiplus,deltachi3 = precession.deltachiroots(kappa=kappa, u=u, chieff=chieff, q=q, chi1=chi1, chi2=chi2)
m = precession.elliptic_parameter(kappa=kappa, u=u, chieff=chieff, q=q, chi1=chi1, chi2=chi2,
                                  precomputedroots=np.stack([deltachiminus, deltachiplus, deltachi3]))
deltachiav = precession.inverseaffine( precession.deltachitildeav(m),  deltachiminus, deltachiplus)
deltachiav

In [ ]:
(deltachiav_-deltachiav)/deltachiav

This is also an example of  how providing the optional quantity `precomputedroots` speeds up the computation.